In [7]:
import ape
from ape.api import ProviderAPI, BlockAPI, TransactionAPI
from ape import Contract
from ape.contracts import ContractInstance
from ape.contracts.base import ContractTransaction, ContractTransactionHandler

 ## Set up ape

In [8]:
USE_ALCHEMY = False
PROVIDER_STRING = "alchemy" if USE_ALCHEMY else "http://localhost:8547"
provider: ProviderAPI = ape.networks.parse_network_choice(f"ethereum:goerli:{PROVIDER_STRING}").push_provider()

INFO: Connecting to existing Erigon node at  http://localhost:8547.


 ### show original `block.base_fee` behavior

In [9]:
block = ape.chain.blocks[-1]
block2 = provider.get_block("latest")
block3 = provider.get_block("pending")
assert block == block2
print(f"block.base_fee  = {getattr(block, 'base_fee'):,.0f} in {block=}")
print(f"block2.base_fee = {getattr(block2, 'base_fee'):,.0f} in {block2=}")
print(f"block3.base_fee = {getattr(block3, 'base_fee'):,.0f} in {block3=}")

block.base_fee  = 735,823,625,899 in block=Block(num_transactions=92, hash=HexBytes('0xb5cbcb6e65a5cb64c4e0988f494605351fc18460810119cbd7c975e16955e7b2'), number=8946801, parent_hash=HexBytes('0xfdd5139783b75c9af9f335765324393ba097e14386a919b77d65cd6eff45aba1'), size=57082, timestamp=1683297504, gas_limit=30000000, gas_used=15068846, base_fee=735823625899, difficulty=0, total_difficulty=10790000)
block2.base_fee = 735,823,625,899 in block2=Block(num_transactions=92, hash=HexBytes('0xb5cbcb6e65a5cb64c4e0988f494605351fc18460810119cbd7c975e16955e7b2'), number=8946801, parent_hash=HexBytes('0xfdd5139783b75c9af9f335765324393ba097e14386a919b77d65cd6eff45aba1'), size=57082, timestamp=1683297504, gas_limit=30000000, gas_used=15068846, base_fee=735823625899, difficulty=0, total_difficulty=10790000)
block3.base_fee = 735,823,625,899 in block3=Block(num_transactions=92, hash=None, number=8946801, parent_hash=HexBytes('0xfdd5139783b75c9af9f335765324393ba097e14386a919b77d65cd6eff45aba1'), size=5708

 ### show old `provider.base_fee` behavior

In [16]:
fee_history = provider.web3.eth.fee_history(1, provider.web3.eth.get_block("latest").number)
last_base_fee, next_base_fee = fee_history["baseFeePerGas"]
print(f"current base fee is         {last_base_fee/1e9:,.2f} 😴")
print(f"base fee FROM THE FUTURE is {next_base_fee/1e9:,.2f} 🤪")
print(f"provider's base fee  is...  {getattr(provider, 'base_fee')/1e9:,.2f} 🤦")

current base fee is         687.68 😴
base fee FROM THE FUTURE is 765.77 🤪
provider's base fee  is...  687.68 🤦


### preparing a new transaction uses the current (latest block) base fee

In [17]:
dai: ContractInstance = Contract("0x11fe4b6ae13d2a6055c8d9cf65c55bac32b5d844")  # goerli DAI
contract_txn = ContractTransaction(address=dai.address,abi=dai.approve.abis[0])
serial_txn: TransactionAPI = contract_txn.serialize_transaction("0x" + "0"*40, 1, gas_limit=int(1e6))
prepped_txn: TransactionAPI = provider.prepare_transaction(serial_txn)
max_fee, max_priority_fee = prepped_txn.max_fee, prepped_txn.max_priority_fee
implied_base_fee = max_fee - max_priority_fee
print(f"base fee = {implied_base_fee/1e9:,.2f}")
print(f"         = max_fee ({max_fee/1e9:,.2f}) - max_priority_fee ({max_priority_fee/1e9:,.2f})")
if implied_base_fee == last_base_fee:
    print(f"this matches LAST base fee {last_base_fee/1e9:,.2f}")
elif implied_base_fee == next_base_fee:
    print(f"this matches NEXT base fee {next_base_fee/1e9:,.2f}")

base fee = 687.68
         = max_fee (689.09) - max_priority_fee (1.41)
this matches **last** base fee 687.68
